# Monkeypox KG


# Package installation is required only once

In [ ]:
#pip install -r packages.txt

In [1]:
import pybel
from tqdm import tqdm
import pandas as pd
from pybel.dsl import Protein
from pybel.dsl import Abundance
from pybel.dsl import Pathology
from pybel.dsl import BiologicalProcess
from pybel.dsl import Population
from pybel.dsl import Gene
from pybel.dsl import Rna
from pybel.dsl import MicroRna
from pybel.dsl import Fragment
import chembl_webresource_client
import openpyxl
import networkx as nx
from pybel.io.jupyter import to_jupyter
import matplotlib.pyplot as plt
from chembl_webresource_client.new_client import new_client
import pubchempy
import pickle
import re
from ByCovidGraph import *
import pubchempy as pcp
import urllib
from urllib.parse import urlparse
import io


C:\Users\reagon.karki\Anaconda3\envs\bycovid_new\lib\site-packages\pybel\struct\operations.py:8: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [ ]:
# drugs against monkeypox from https://www.jglobalbiosecurity.com/articles/10.31646/gbio.12

mpox = 'CHEMBL152 CHEMBL203321 CHEMBL1257073 CHEMBL220610 CHEMBL221054 CHEMBL1257073 CHEMBL375768 CHEMBL220150'.split()
mpox

In [ ]:
mpox

In [ ]:
#reading file from pubchem for monkeypox 
#https://pubchem.ncbi.nlm.nih.gov/taxonomy/10244

mpox_pchem = pd.read_csv('data/normalized_data/TaxID_10244_bioactivity.csv')
mpox_pchem_cid = mpox_pchem['cid']
mpox_pchem_cid = list(set(mpox_pchem_cid))
len(mpox_pchem_cid)

In [ ]:
mpox_pchem_cid

In [ ]:
chembl_list = cid2chembl(mpox_pchem_cid)

In [ ]:
chembl_list

In [ ]:
#convert pubchem cids to chembl ids

cid2chembl_dict = []

for id in mpox_pchem_cid:
    #if id != 'None':
    print(id)
    GetChembl = {}
    c=pcp.Compound.from_cid(id)
    #print(id)
    syn = c.synonyms
    for s in syn:
        #if s.startswith('SCHEMBL'):
            #GetChembl['SCHEMBL'] = s
        if s.startswith('CHEMBL'):
            print(s)
            GetChembl['CHEMBL'] = s

    cid2chembl_dict.append(GetChembl)
        

In [ ]:
cid2chembl_dict

In [ ]:
chemblid = []

for id in cid2chembl_dict:
    #chemblid.append()
    #print(id.values())
    chembl = list(id.values())
    #print(chembl[0])
    chemblid.append(chembl[0])
    

In [ ]:
chemblid = mpox+chembl_list
len(chemblid)

In [ ]:
chemblid = set(chemblid)
chemblid = list(chemblid)
len(chemblid)

In [ ]:
pchem_mech = RetMech(chemblid)


In [ ]:
pchem_dis = RetDrugInd(chemblid)

In [ ]:
pchem_act = RetAct(chemblid,0)

In [123]:
chemid2 = Ret_chembl_protein(pchem_act) + Ret_chembl_protein(pchem_mech)
chemid2

NameError: name 'pchem_act' is not defined

In [125]:
chemid2

['CHEMBL2754', 'CHEMBL2413', 'CHEMBL614548', 'CHEMBL3091']

In [ ]:
chem2uni = chembl2uniprot(chemid2,0)
chem2uni

In [ ]:
chem2uni = chembl2uniprot(chemid2,0)
chem2uni

In [ ]:
chem2uni

In [ ]:
pchem_act_new= chembl2gene2path(chem2uni,pchem_act)
pchem_act_new

In [ ]:
pchem_act

In [ ]:
mpox_graph = pybel.BELGraph(name='Monkeypox Graph')

In [ ]:
mpox_graph = chem2moa_rel_2(pchem_mech,mpox_graph)
mpox_graph = chem2dis_rel(pchem_dis,mpox_graph)
mpox_graph = chem2act_rel_2(pchem_act_new,mpox_graph)
mpox_graph = chem2gene2path_rel(chem2uni,mpox_graph)
to_jupyter(mpox_graph)

In [ ]:
mpox_graph.summarize()

# Working with monkeypox proteins from uniprot


In [ ]:
mpox_prot_df = pd.read_excel('data/normalized_data/uniprot-taxonomy Monkeypox+virus+[10244] -filtered-reviewed yes.xlsx')

In [ ]:
mpox_prot= list(mpox_prot_df['Entry'])
mpox_prot

In [ ]:
mpox_prot_ext = ExtractFromUniProt(mpox_prot)

In [ ]:
mpox_prot_ext

In [ ]:
str(mpox_prot_ext['Q8V4S4']['Gene'])

In [ ]:
def uniprot_rel(named_uprotList,itmpGraph):
    
    for item in named_uprotList:
        #print(named_uprotList[item]['Function'].keys())
        fun=list(named_uprotList[item]['Function'].keys())
        bp = list(named_uprotList[item]['BioProcess'].keys())
        for f in fun:
            if str(named_uprotList[item]['Gene']) != 'nan' && !isinstance(named_uprotList[item]['Gene'],dict) :
                itmpGraph.add_association(Protein(namespace='MP',name=named_uprotList[item]['Gene']),MicroRna(namespace='Function',name=f),
                                         citation='UniProt database',evidence='UniProt query')
            else:
                itmpGraph.add_association(Protein(namespace='MP',name=item),MicroRna(namespace='Function',name=f),
                                         citation='UniProt database',evidence='UniProt query')
                
        
        for b in bp:
            if str(named_uprotList[item]['Gene']) != 'nan' && !isinstance(named_uprotList[item]['Gene'],dict):
                itmpGraph.add_association(Protein(namespace='MP',name=named_uprotList[item]['Gene']),BiologicalProcess(namespace='BioPro',name=b),
                                         citation='UniProt database',evidence='UniProt query')
            else:
                itmpGraph.add_association(Protein(namespace='MP',name=item),MicroRna(namespace='Function',name=b),
                                         citation='UniProt database',evidence='UniProt query')
        
    return(itmpGraph)
        

In [ ]:
#mpox_graph = pybel.BELGraph(name='Monkeypox Graph')

In [ ]:
mpox_graph = uniprot_rel(mpox_prot_ext,mpox_graph)

In [ ]:
mpox_graph = uniprot_rel(mcards_prot,mpox_graph)

In [ ]:
mpox_graph.summarize()

In [ ]:
to_jupyter(mpox_graph)

# Genes from Malacards


In [ ]:
mpox_genes = 'IFNA1 C4A C4B CCL26 KLRC3 CD4 CD8A OR10G6 LINCO1285 KLRK1 CD55 CYB5R3 CD46 IRF3'.split(" ")
mpox_genes

In [2]:
mpox_genes = pd.read_excel('data/normalized_data/malacardsGenes.xlsx')

In [3]:
mpox_genes.columns

Index(['yourlist:M20220620A084FC58F6BBA219896F365D15F2EB44516BBDJ', 'Entry',
       'Entry name', 'Status', 'Protein names', 'Gene names', 'Organism',
       'Gene names  (primary )'],
      dtype='object')

In [4]:
mpox_prot = list(mpox_genes['Entry'])
mpox_prot

['P0C0L4',
 'P0C0L5',
 'Q9Y258',
 'Q07444',
 'P01730',
 'P01732',
 'Q8NH81',
 'P26718',
 'P08174',
 'P00387',
 'P15529',
 'Q14653',
 'P01562']

In [5]:
mcards_prot = ExtractFromUniProt(mpox_prot)

https://www.uniprot.org/uniprot/P0C0L4.txt
P0C0L4
https://www.uniprot.org/uniprot/P0C0L5.txt
P0C0L5
https://www.uniprot.org/uniprot/Q9Y258.txt
Q9Y258
https://www.uniprot.org/uniprot/Q07444.txt
Q07444
https://www.uniprot.org/uniprot/P01730.txt
P01730
https://www.uniprot.org/uniprot/P01732.txt
P01732
https://www.uniprot.org/uniprot/Q8NH81.txt
Q8NH81
https://www.uniprot.org/uniprot/P26718.txt
P26718
https://www.uniprot.org/uniprot/P08174.txt
P08174
https://www.uniprot.org/uniprot/P00387.txt
P00387
https://www.uniprot.org/uniprot/P15529.txt
P15529
https://www.uniprot.org/uniprot/Q14653.txt
Q14653
https://www.uniprot.org/uniprot/P01562.txt
P01562


In [1]:
mcards_prot = ExtractFromUniProt(mpox_prot)

NameError: name 'ExtractFromUniProt' is not defined

In [ ]:
mcards_prot

In [ ]:
#Exporting the graph

#to cytoscape compatible graphml 
pybel.to_graphml(mpox_graph,'data/normalized_data/Monkeypox_KG.graphml')

#to regular BEL format
pybel.dump(mpox_graph,'data/normalized_data/Monkeypox_KG.bel')

#to neo4j
pybel.to_csv(mpox_graph,'data/normalized_data/Monkeypox_KG.csv')

#to sif
pybel.to_sif(mpox_graph,'data/normalized_data/Monkeypox_KG.sif')

# Convert uniprot to chembl

In [ ]:
from chembl_webresource_client.new_client import new_client

target = new_client.target
gene_name = 'P01730'
res = target.filter(target_synonym__icontains=gene_name)
for i in res:
    print(i)

In [ ]:
query = ' '.join(mpox_prot)
query

In [ ]:
import urllib.parse
import urllib.request

url = 'https://www.uniprot.org/uploadlists/'

query = 'P0C0L4 P0C0L5 Q9Y258 Q07444 P01730 P01732 Q8NH81 P26718 P08174 P00387 P15529 Q14653 P01562'
params = {
'from': 'ACC+ID',
'to': 'CHEMBL_ID',
'format': 'tab',
'query': query
}

data = urllib.parse.urlencode(params)
data = data.encode('utf-8')
req = urllib.request.Request(url, data)
with urllib.request.urlopen(req) as f:
   response = f.read()
print(response.decode('utf-8'))

In [ ]:
query = 'P0C0L4 P0C0L5 Q9Y258 Q07444 P01730 P01732 Q8NH81 P26718 P08174 P00387 P15529 Q14653 P01562'.split(' ')
query

In [ ]:
query = ExtractFromUniProt(query)

In [ ]:
query

In [ ]:
targets_api  = new_client.target


In [6]:

def _get_target_data(protein_list: list, organism: str):
    """Get chemical for target data from ChEMBL"""
    df_data = []

    target = new_client.target
    activity = new_client.activity

    for protein in protein_list:
        if pd.isna(protein):
            continue
        try:
            prot_data = [target.search(protein)[0]]

            # Search for protein with same synonym
            if prot_data == [None]:
                prot_data = target.filter(
                    target_synonym__icontains=protein, target_organism__istartswith=organism
                ).only(['target_chembl_id', 'target_pref_name', 'molecule_chembl_id', 'molecule_pref_name'])
        except HttpBadRequest:
            print(f'Cannot search for {protein} due to chembl error')
            continue

        # No results found
        if not prot_data:
            continue

        for prot in tqdm(prot_data, f'Analying data for {protein}'):
            # Absence of chembl id
            if not prot['target_chembl_id']:
                continue

            prot_activity_data = activity.filter(
                target_chembl_id=prot['target_chembl_id'],
                assay_type_iregex='(B|F)',
            ).only([
                'pchembl_value', 'molecule_chembl_id', 'activity_id', 'target_pref_name', 'molecule_pref_name'
            ])

            if len(prot_activity_data) < 1:
                continue

            for i in prot_activity_data:
                tmp = {}

                if i['pchembl_value'] is None:
                    continue

                pchembl_val = i['pchembl_value']

                if float(pchembl_val) < 6:
                    tmp['activity'] = 'inhibitor'
                else:
                    tmp['activity'] = 'activator'

                tmp['protein_symbol'] = protein
                tmp['protein_name'] = i['target_pref_name']
                tmp['aid'] = str(i['activity_id'])
                tmp['chembl_id'] = i['molecule_chembl_id']
                tmp['compound_name'] = i['molecule_pref_name'].capitalize() if i['molecule_pref_name'] else ''
                df_data.append(tmp)

    # Merge duplicated protein-chemical entries into one
    df = pd.DataFrame()

    for idx, row in tqdm(enumerate(df_data), total=len(df_data), desc='Preparing data'):
        if idx == 0:
            df = df.append(row, ignore_index=True)
        else:
            _in_df = df.loc[
                (df['protein_symbol'] == row['protein_symbol']) & (df['chembl_id'] == row['chembl_id'])
            ]

            if _in_df.empty:
                df = df.append(row, ignore_index=True)
            else:
                row_index = _in_df.index

                # Check existing citations
                existing_assays = set(df.loc[row_index, 'aid'].values[0].split(' | '))
                old_count = len(existing_assays)
                existing_assays.add(row['aid'])
                new_count = len(existing_assays)

                # Check if new citation added, if yes - add respective data
                if old_count < new_count:
                    df.loc[row_index, 'aid'] = ' | '.join(existing_assays)
    df = df[['activity', 'protein_symbol', 'protein_name', 'aid', 'chembl_id', 'compound_name']]
    return df


In [7]:
def target_list_to_chemical(
    proteins: list,
    organism: str = 'Homo sapiens',
    output_dir: str = ''
) -> None:
    """Extract chemical information on list of targets
    Usage:
    >> target_list_to_chemical(proteins=['RIPK'])
    """

    df = _get_target_data(protein_list=proteins, organism=organism)
    #os.makedirs(output_dir, exist_ok=True)
    #df.to_csv(os.path.join(output_dir, 'chemical_annotated.csv'), sep='\t', index=False)
    return(df)


In [8]:
tar2chem = target_list_to_chemical(mpox_prot)

Preparing data:   0%|          | 0/38 [00:00<?, ?it/s]C:\Users\reagon.karki\AppData\Local\Temp\ipykernel_780\3358757808.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
Preparing data:   3%|▎         | 1/38 [00:00<00:14,  2.64it/s]C:\Users\reagon.karki\AppData\Local\Temp\ipykernel_780\3358757808.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\reagon.karki\AppData\Local\Temp\ipykernel_780\3358757808.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\reagon.karki\AppData\Local\Temp\ipykernel_780\3358757808.py:74: FutureWarning: The frame.append method is deprecated and will be remov

C:\Users\reagon.karki\AppData\Local\Temp\ipykernel_780\3358757808.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\reagon.karki\AppData\Local\Temp\ipykernel_780\3358757808.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
Preparing data:  87%|████████▋ | 33/38 [00:00<00:00, 55.62it/s]C:\Users\reagon.karki\AppData\Local\Temp\ipykernel_780\3358757808.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\reagon.karki\AppData\Local\Temp\ipykernel_780\3358757808.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat

In [ ]:
tar2chem

In [9]:
list(tar2chem['chembl_id'])

['CHEMBL506605',
 'CHEMBL525803',
 'CHEMBL3143464',
 'CHEMBL3138360',
 'CHEMBL3138345',
 'CHEMBL3138187',
 'CHEMBL3138374',
 'CHEMBL3138095',
 'CHEMBL3138378',
 'CHEMBL3138119',
 'CHEMBL231224',
 'CHEMBL267655',
 'CHEMBL388209',
 'CHEMBL230799',
 'CHEMBL267658',
 'CHEMBL388208',
 'CHEMBL267657',
 'CHEMBL230798',
 'CHEMBL388207',
 'CHEMBL230796',
 'CHEMBL1852155',
 'CHEMBL1819181',
 'CHEMBL4783030',
 'CHEMBL4755141',
 'CHEMBL4783761',
 'CHEMBL4750160',
 'CHEMBL4743003',
 'CHEMBL4782080',
 'CHEMBL4746252',
 'CHEMBL4747494',
 'CHEMBL4758659',
 'CHEMBL4750983',
 'CHEMBL4786809',
 'CHEMBL4748787',
 'CHEMBL4785236',
 'CHEMBL4791666',
 'CHEMBL4525466',
 'CHEMBL4471585']

In [10]:
tar2chem_act = RetAct(list(tar2chem['chembl_id']),0)

1
[{'assay_chembl_id': 'CHEMBL655882', 'assay_type': 'B', 'bao_label': 'single protein format', 'pchembl_value': '6.89', 'target_chembl_id': 'CHEMBL2754', 'target_organism': 'Homo sapiens', 'type': 'IC50', 'value': '0.13'}]
2
[{'assay_chembl_id': 'CHEMBL655882', 'assay_type': 'B', 'bao_label': 'single protein format', 'pchembl_value': '6.89', 'target_chembl_id': 'CHEMBL2754', 'target_organism': 'Homo sapiens', 'type': 'IC50', 'value': '0.13'}]
3
[{'assay_chembl_id': 'CHEMBL655882', 'assay_type': 'B', 'bao_label': 'single protein format', 'pchembl_value': '6.89', 'target_chembl_id': 'CHEMBL2754', 'target_organism': 'Homo sapiens', 'type': 'IC50', 'value': '0.13'}]
4
[]
5
[{'assay_chembl_id': 'CHEMBL659225', 'assay_type': 'F', 'bao_label': 'cell-based format', 'pchembl_value': '6.30', 'target_chembl_id': 'CHEMBL614548', 'target_organism': 'Homo sapiens', 'type': 'EC50', 'value': '0.5'}, {'assay_chembl_id': 'CHEMBL659282', 'assay_type': 'F', 'bao_label': 'cell-based format', 'pchembl_valu

In [12]:
tar2chem_mech = RetMech(list(tar2chem['chembl_id']))

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [11]:
tar2chem_dis = RetDrugInd(list(tar2chem['chembl_id']))

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [13]:
chemid2 = Ret_chembl_protein(tar2chem_act) + Ret_chembl_protein(tar2chem_mech)

In [14]:
chem2uni = chembl2uniprot(chemid2,0)

old 3
newLen 3
0


In [99]:
chem2uni

{'CHEMBL2754': [{'xref_id': 'R-HSA-1462054',
   'xref_name': 'Alpha-defensins.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-167590',
   'xref_name': 'Nef Mediated CD4 Down-regulation.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-173107',
   'xref_name': 'Binding and entry of HIV virion.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-180534',
   'xref_name': 'Vpu mediated degradation of CD4.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-202424',
   'xref_name': 'Downstream TCR signaling.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-202427',
   'xref_name': 'Phosphorylation of CD3 and TCR zeta chains.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-202430',
   'xref_name': 'Translocation of ZAP-70 to Immunological synapse.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-202433',
   'xref_name': 'Generation of second messenger molecules.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-389948',
   'xref_name': 'PD-1 signaling.',
 

In [103]:
protList = []
for item in chem2uni['CHEMBL2754']:
    #print(type(item))
    if 'accession' in item:
        print(item['accession'])

P01730


In [118]:
protList = []
for key in chem2uni.keys():
    #print(chem2uni[key])
    for item in chem2uni[key]:
        if 'accession' in item:
            protList.append(item['accession'])
            #print(item['accession'])

In [119]:
protList 

['P01730', 'P32246', 'Q99523']

In [121]:
protList = ExtractFromUniProt(protList)

https://www.uniprot.org/uniprot/P01730.txt
P01730
https://www.uniprot.org/uniprot/P32246.txt
P32246
https://www.uniprot.org/uniprot/Q99523.txt
Q99523


In [122]:
protList

{'P01730': {'Disease': {'Immunodeficiency 79 (IMD79)': '619238'},
  'Reactome': {'Alpha-defensins': 'R-HSA-1462054',
   'Nef Mediated CD4 Down-regulation': 'R-HSA-167590',
   'Binding and entry of HIV virion': 'R-HSA-173107',
   'Vpu mediated degradation of CD4': 'R-HSA-180534',
   'Downstream TCR signaling': 'R-HSA-202424',
   'Phosphorylation of CD3 and TCR zeta chains': 'R-HSA-202427',
   'Translocation of ZAP-70 to Immunological synapse': 'R-HSA-202430',
   'Generation of second messenger molecules': 'R-HSA-202433',
   'PD-1 signaling': 'R-HSA-389948',
   'Other interleukin signaling': 'R-HSA-449836',
   'Cargo recognition for clathrin-mediated endocytosis': 'R-HSA-8856825',
   'Clathrin-mediated endocytosis': 'R-HSA-8856828'},
  'Function': {'coreceptor activity': 'GO:0015026',
   'enzyme binding': 'GO:0019899',
   'extracellular matrix structural constituent': 'GO:0005201',
   'identical protein binding': 'GO:0042802',
   'interleukin-16 binding': 'GO:0042011'},
  'BioProcess': {

In [98]:
'accession' in chem2uni['CHEMBL2754']

True

In [47]:
'accession' in chem2uni['CHEMBL2754']

False

In [16]:
pchem_act_new= chembl2gene2path(chem2uni,tar2chem_act)

In [18]:
pchem_act_new

{'CHEMBL506605': [{'assay_chembl_id': 'CHEMBL655882',
   'assay_type': 'B',
   'bao_label': 'single protein format',
   'pchembl_value': '6.89',
   'target_chembl_id': 'CHEMBL2754',
   'target_organism': 'Homo sapiens',
   'type': 'IC50',
   'value': '0.13',
   'Protein': 'CD4'}],
 'CHEMBL525803': [{'assay_chembl_id': 'CHEMBL655882',
   'assay_type': 'B',
   'bao_label': 'single protein format',
   'pchembl_value': '6.89',
   'target_chembl_id': 'CHEMBL2754',
   'target_organism': 'Homo sapiens',
   'type': 'IC50',
   'value': '0.13',
   'Protein': 'CD4'}],
 'CHEMBL3143464': [{'assay_chembl_id': 'CHEMBL655882',
   'assay_type': 'B',
   'bao_label': 'single protein format',
   'pchembl_value': '6.89',
   'target_chembl_id': 'CHEMBL2754',
   'target_organism': 'Homo sapiens',
   'type': 'IC50',
   'value': '0.13',
   'Protein': 'CD4'}],
 'CHEMBL3138345': [{'assay_chembl_id': 'CHEMBL659225',
   'assay_type': 'F',
   'bao_label': 'cell-based format',
   'pchembl_value': '6.30',
   'target_

In [19]:
#mpox_graph = chem2moa_rel_2(pchem_mech,mpox_graph)
#mpox_graph = chem2dis_rel(pchem_dis,mpox_graph)
mpox_graph = chem2act_rel_2(pchem_act_new,mpox_graph)
mpox_graph = chem2gene2path_rel(chem2uni,mpox_graph)
to_jupyter(mpox_graph)

NameError: name 'mpox_graph' is not defined

In [ ]:
# Get target information from ChEMBL but restrict it to specified values only

#uniprot_id = "P01562"
targets = targets_api.get(target_components__accession="Q14653").only(
    "target_chembl_id", "organism", "pref_name", "target_type"
)

targets[0]

In [ ]:
mpox_prot2chembl = []
for item in mpox_prot:
    targets = targets_api.get(target_components__accession=item).only(
    "target_chembl_id", "organism", "pref_name", "target_type")
    mpox_prot2chembl.append(targets[0])

In [ ]:
mpox_prot2chembl

In [ ]:
named_mpox_prot2chembl = dict(zip(mpox_prot,mpox_prot2chembl))

In [ ]:
named_mpox_prot2chembl = {k: v for k, v in named_mpox_prot2chembl.items() if v}

In [ ]:
named_mpox_prot2chembl

In [ ]:
getchem = []
for item in named_mpox_prot2chembl:
    getchem.append(named_mpox_prot2chembl[item]['target_chembl_id'])
getchem

In [ ]:
RetAct(getchem,0)

In [ ]:
bioactivities = bioactivities_api.filter(
    target_chembl_id='CHEMBL3713007', type="IC50", relation="=", assay_type="B|F"
).only(
    "activity_id",
    "assay_chembl_id",
    "assay_description",
    "assay_type",
    "molecule_chembl_id",
    "type",
    "standard_units",
    "relation",
    "standard_value",
    "target_chembl_id",
    "target_organism",
)

In [ ]:
bioactivities

In [ ]:
test = chembl2uniprot(getchem,0)

# Philip


In [ ]:
chem = ['58-74-2','486-47-5','14009-24-6']

In [ ]:
def cas2cid(cas):
    cid_clean = []
    #cid = [pcp.get_compounds(id,'name') for id in cas]
    for id in cas:
        print(id)
        #retrieves cid for given cas 
        cid = pcp.get_compounds(id,'name')
        print(cid)
        #strip unwanted strings to get cid 
        filter_cid = re.findall('\((.+?)\)',str(cid))
        #print(filter_cid[0])
        if len(filter_cid) > 0:
            cid_clean.append(filter_cid[0])
        else:
            cid_clean.append('None')

    return(cid_clean)

In [ ]:
chem2cid = cas2cid(chem)


In [ ]:
chem2cid

In [ ]:

cid2chembl_dict = []
i=0
for id in chem2cid:
    #if id != 'None':
    print(id)
    GetChembl = {}
    c=pcp.Compound.from_cid(id)
    print("working with:"+id)
    syn = c.synonyms
    for s in syn:

        if s.startswith('SCHEMBL'):
            GetChembl['SCHEMBL'] = s
        if s.startswith('CHEMBL'):
            GetChembl['CHEMBL'] = s
            
    i=i+1
    print(i)

    cid2chembl_dict.append(GetChembl)
        

In [ ]:
cid2chembl_dict

In [ ]:
cid2chembl_dict = dict(zip(list(chem2cid), cid2chembl_dict))
cid2chembl_dict

In [ ]:
cid_prop = []
i=0
for id in chem2cid:
    propOfInt = ['canonical_smiles','exact_mass','h_bond_acceptor_count','h_bond_donor_count','xlogp','rotatable_bond_count',
            'tpsa','heavy_atom_count','inchi','inchikey','isomeric_smiles','molecular_formula','molecular_weight']
    c = pcp.Compound.from_cid(id)
    i+=1
    print(i)
    c = c.to_dict(properties=propOfInt)
    cid_prop.append(c)

In [ ]:
cid_prop

In [ ]:
for k in cid2chembl_dict:
     cid2chembl_dict[k].update(cid_prop.get(k, {}))

In [ ]:
chembl = ['CHEMBL19224','CHEMBL1555736','CHEMBL551978']

In [ ]:
getAct = RetAct(chembl,0)

In [ ]:
getMech = RetMech(chembl)

In [ ]:
getDis = RetDrugInd(chembl)

In [ ]:
chemid2 = Ret_chembl_protein(getAct) + Ret_chembl_protein(getMech)

In [ ]:
chemid2

In [ ]:
chem2uni = chembl2uniprot(chemid2,0)

In [ ]:
pchem_act_new= chembl2gene2path(chem2uni,getAct)

In [ ]:
pchem_act_new

In [ ]:
mpox_graph = pybel.BELGraph(name='Test Graph')

In [ ]:
mpox_graph = chem2moa_rel_2(getMech,mpox_graph)
mpox_graph = chem2dis_rel(getDis,mpox_graph)
mpox_graph = chem2act_rel_2(pchem_act_new,mpox_graph)
mpox_graph = chem2gene2path_rel(chem2uni,mpox_graph)
to_jupyter(mpox_graph)